James Caldwell <br>
5/6/25 <br>
Data Request 604 <br>

This scrip imports a dataset of studetns in the Hereford College. It then calculates the percentage and counts of the students by Race, First generation status, academic year, and low income status. It also determines the number of students that were on the roster but not found in the current academic year.

In [3]:
import pandas as pd
import numpy as np
import os

In [4]:
df = pd.read_excel('Hereford College 2024-2025.xlsx', sheet_name='Raw Data')

qlik_emails = df['Email Address UVA']
roster_list = pd.read_excel('Hereford College 2024-2025.xlsx', sheet_name='Hereford College 2024-2025 rost')
missing = [email for email in roster_list['Email Address UVA'].to_list() if email not in qlik_emails.to_list()]

# for email in missing:
    # print(email + ',')


In [47]:
df['Academic Year'] = df['Student Group Academic Year'].map({'First-Year':1, 
                                        'Second-Year':2,
                                        'Third-Year':3,
                                        'Fourth-Year':4})

In [48]:
df.sort_values(by=['Email Address UVA','Academic Year'], ascending=True, inplace=True)

In [49]:
df.drop_duplicates(subset=['Email Address UVA'], keep='last', inplace=True)

In [50]:
df.drop(columns=['Email Address UVA'], inplace=True)

In [51]:
def describe_with_counts(df):
    result = {}
    for col in df.columns:
        counts = df[col].value_counts(dropna=False)
        percents = df[col].value_counts(normalize=True, dropna=False) * 100
        result[col] = pd.DataFrame({
            'Count': counts,
            'Percent': percents.round(2)
        })
    return result

summaries = describe_with_counts(df)
for col, summary in summaries.items():
    print(f"Column: {col}")
    print(summary)
    print("-" * 40)

# Save to Excel
with pd.ExcelWriter('column_summaries.xlsx') as writer:
    for col, summary in summaries.items():
        # Sheet names can't be longer than 31 characters
        sheet_name = str(col)[:31]
        summary.to_excel(writer, sheet_name=sheet_name)

Column: IPEDS Race
                                  Count  Percent
IPEDS Race                                      
White                                64    31.84
Asian                                40    19.90
Black or African American            35    17.41
Non-Resident Alien                   22    10.95
Hispanic                             21    10.45
Multi-Race                           12     5.97
Race and Ethnicity Unknown            6     2.99
American Indian or Alaska Native      1     0.50
----------------------------------------
Column: First Generation
                  Count  Percent
First Generation                
Other               141    70.15
First Generation     60    29.85
----------------------------------------
Column: Reported Low Income Flag
                          Count  Percent
Reported Low Income Flag                
Missing Data                 97    48.26
Not Low Income               65    32.34
Low Income                   39    19.40
--------------